In [4]:
import numpy as np
import os
import tifffile
import cv2
from os.path import join, isfile, exists
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn as nn
import os
from PIL import Image

In [10]:
# Define the dataset class
class MyDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        files = [f for f in os.listdir(root_dir) if isfile(join(root_dir, f))]
        for file in files:
            file_path = os.path.join(root_dir, file)
            print(file_path)
            if file_path.endswith(".png"):
                
                img = Image.open(file_path).convert("RGB")
                # self.images.append(img)
                name = file.split("_")[0]
                label = label_map.get(name, -1)
                if label != -1:
                    self.images.append(img)
                    self.labels.append(label)
                # self.labels.append(lable_map[name])
            elif file_path.endswith(".tif"):
                image_array = tifffile.imread(file_path)
                img_rescaled = 255 * (image_array - image_array.min()) / (image_array.max() - image_array.min())
                img_col = cv2.applyColorMap(img_rescaled.astype(np.uint8), cv2.COLORMAP_DEEPGREEN)
                img = Image.fromarray(img_col)
                img = img.convert("RGB")
                # self.images.append(img)

                name = file.split("_")[0]
                label = label_map.get(name, -1)
                if label != -1:
                    self.images.append(img)
                    self.labels.append(label)
                # self.labels.append(lable_map[name])
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        # print(np.array(img))
        if self.transform is not None:
            img = self.transform(img)
        return img, label


In [17]:
input_dir = "../images"
output_dir = "../images/png"
label_map = {"BMP4" :0, "CHIR": 1, "DS": 2, "DSandCHIR": 3,  "WT": 4}
# Define the input shape of the images
input_shape = (3, 224, 224)
# Define the number of classes
num_classes = 5
# Load the pre-trained GoogleNet model
model = models.googlenet(pretrained=True)

# if I dont need to train, use this
model = nn.Sequential(*list(model.children())[:-1])
#else use this
# Freeze the weights of all layers except the last fully connected layer
# for param in model.parameters():
#     param.requires_grad = False
# # Remove the last fully connected layer
# model.fc = nn.Identity()
# model.fc.requires_grad = True
# Replace the last fully connected layer with a new layer that has `num_classes` output units
# model.fc = nn.Linear(1024, num_classes)
# Define the loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

In [7]:
# Define the data transformations
# transform = transforms.Compose([
#     transforms.Resize(input_shape[1:]),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
# ])
# # Load the dataset
# dataset = MyDataset("../images/png", transform=transform)
# image0, label0 = dataset.__getitem__(0)
# # print(image0)
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

../images/png\BMP4_22_20_t260_c002.png
../images/png\CHIR_22_40_t140_c002.png
../images/png\DSandCHIR_26_14_t150_c002.png
../images/png\DS_26_04_t060_c002.png
../images/png\WT_22_02_t020_c002.png


In [18]:
# Define the data transformations
transform = transforms.Compose([
    transforms.Resize(input_shape[1:]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
# Load the dataset
dataset = MyDataset("../images", transform=transform)
image0, label0 = dataset.__getitem__(0)
# print(image0)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


../images\BMP4_22_20_t260_c002.tif
../images\CHIR_22_40_t140_c002.tif
../images\DSandCHIR_26_14_t150_c002.tif
../images\DS_26_04_t060_c002.tif
../images\WT_22_02_t020_c002.tif


In [21]:
# Define an empty list to store the extracted features
features = []
# Extract features from the images
with torch.no_grad():
    for images, _ in dataloader:
        features_batch = model(images)
        features.append(features_batch)
# Concatenate the extracted features
features = torch.cat(features)
print(features.shape)
# print(features[0])

torch.Size([5, 1024, 1, 1])


In [22]:
# Train the model
# num_epochs = 50
# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for i, data in enumerate(dataloader, 0):
#         # print(i)
#         inputs, labels = data
#         # print(inputs, labels)

#         # Zero the parameter gradients
#         optimizer.zero_grad()

#         # Forward + backward + optimize
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         # Print statistics
#         running_loss += loss.item()
#         # if i % 10 == 9:
#         print(f"[Epoch {epoch+1}, Batch {i+1}] Loss: {running_loss/10:.3f}")
#         running_loss = 0.0
        
            

In [23]:
## RNN part

In [24]:
# Reshape the extracted features
sequence_length = features.size(0)
num_features = features.size(1)
reshaped_features = features.view(sequence_length, num_features)

# Design the RNN architecture
hidden_size = 256  # Number of hidden units in the RNN layer

In [25]:
# Set up the RNN model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(RNNModel, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        output, _ = self.rnn(x)
        output = output[:, -1, :]  # Get the output at the last time step
        output = self.fc(output)
        return output

In [ ]:
class MyDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.classes = []
        self.image_paths = []
        
        folders = os.listdir(root_dir)
        
        for folder_name in folders:
            folder_path = os.path.join(root_dir, folder_name)
            
            if os.path.isdir(folder_path):
                class_name = folder_name.split("_")[0]  # Extract the classname from the folder name
                class_idx = len(self.classes)
                
                self.classes.append(class_name)
                
                image_files = os.listdir(folder_path)
                
                for image_file in image_files:
                    image_path = os.path.join(folder_path, image_file)
                    self.image_paths.append(image_path)
        
        self.transform = transform
    
    def __getitem__(self, index):
        image_path = self.image_paths[index]
        class_name = self.classes.index(image_path.split("/")[-2])  # Get the classname from the image path
        image = Image.open(image_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
        
        return image, class_name
    
    def __len__(self):
        return len(self.image_paths)


In [ ]:
dataset = MyDataset("../images/classes", transform=transform)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
input_size = 3  # Assuming RGB images
hidden_size = 256
num_classes = 5

rnn_model = RNNModel(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn_model.parameters(), lr=0.001)


In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    rnn_model.train()  # Set the model to train mode
    
    for images in train_loader:
        optimizer.zero_grad()
        
        # Reshape the images tensor to match the input shape of the RNN
        sequence_length = images.size(1)
        images = images.view(-1, sequence_length, input_size, 224, 224)
        
        # Forward pass
        outputs = rnn_model(images)
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    
    # Perform validation after each epoch if desired
    
# Perform testing after training
rnn_model.eval()  # Set the model to evaluation mode
# Run inference on the test
